# Exemple de finetuning d'un modèle BERT sur la classification des avis du site Allociné

[Ouvrir dans Google colab](https://colab.research.google.com/github/Herve07h22/nlp-fine-tuning/blob/main/allocine.ipynb)

In [3]:
%pip install datasets transformers

You should consider upgrading via the '/Users/herve/Documents/projets/temporaire/nlp-fine-tuning/.env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

In [5]:
raw_datasets = load_dataset("allocine")
raw_datasets.shape

Reusing dataset allocine_dataset (/Users/herve/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb)
100%|██████████| 3/3 [00:00<00:00, 68.73it/s]


{'train': (160000, 2), 'validation': (20000, 2), 'test': (20000, 2)}

## aperçu du dataset "Allociné"

5 commentaires positifs

In [6]:
positive_samples = raw_datasets["train"].shuffle(seed=42).filter(lambda example: example['label']==1)
positive_samples['review'][:5]

Loading cached shuffled indices for dataset at /Users/herve/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb/cache-fce97a8856f0e508.arrow
Loading cached processed dataset at /Users/herve/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb/cache-ce9dae5424c5b159.arrow


['Encore un film iranien. Au fil des films qui arrivent en France, cette société islamique que nous ne comprenons pas, nous livre ses petits secrets et ses incohérences. Par exemple le changement de sexe est autorisé et même remboursé par l’état, un comble pour un pays dont la femme est considérée comme impure (voir le film iranien de Mehran Tamadon) et où l’homosexualité peut conduire à la peine capitale. C’est l’histoire de deux femmes (deux actrices formidables), l’une fait le taxi pour payer la sortie de prison de son mari, l’autre n’aspire qu’à une seul chose, devenir un homme. Ces deux femmes vont se rencontrer, ne pas se comprendre pour finalement s’entraider. Un très beau film avec une très forte morale sur la condition humaine et la religion. 4 étoiles',
 'Grâce à sa réalisation d\'une remarquable qualité, "April Snow" parvient à nous envouter dès la première minute. Certes, certains pourront trouver ça un peu scolaire parfois, mais on ne peut nier l\'efficacité de chaque plan

5 commentaires négatifs

In [7]:
positive_samples = raw_datasets["train"].shuffle(seed=42).filter(lambda example: example['label']==0)
positive_samples['review'][:5]

Loading cached shuffled indices for dataset at /Users/herve/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb/cache-fce97a8856f0e508.arrow
Loading cached processed dataset at /Users/herve/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb/cache-87a99b9c87b7cc5d.arrow


["Ray Milland qui a tourné dans presque 200 films s'est amusé à en réaliser 5.Celui est est le plus connu mais pas le meilleur. Il y a certes quelques séquences réussies mais beaucoup trop qui semblent fabriquées ,les comédiens manquent d'aisance et l'acteur lui même qui sait être excellent avec Hitchcock,Lang,Wilder, et surtout Farrow (the big clock-1948) a du mal à se diriger lui même. L'ambiance est ce qu'il y a de mieux .Imaginer une guerre atomique sans la voir était un pari réussi mais la volonté de trop montrer le mauvais coté de la nature humaine est bien trop excessive et démonstrative. Le contre exemple représenté par la femme du héros finit par agacer. D'autre part,un petit peu plus et nous étions dans un film d'horreur ,ce qui n'était pas le propos .A l'évidence Milland avait d'autres ambitions mais elles ne furent pas tenues. Le film a vieilli ,ce qui n'est jamais bon ,alors qu'avec plus de talent cinématographique il pourrait être encore précurseur puisque ces événements 

## Pre-processing du dataset

Pour l'entraînement du modèle de classification, on utiliser un `Trainer` de HuggingFace. Il utilise un dataset comportant 2 features :
- le label (0 ou 1)
- les input_ids tokenisés
- les attention_mask (1 si le mot est présent, 0 sinon)

On va donc pré-processer notre dataset avec le Tokeniser du modèle

In [8]:
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

tokenized_datasets.shape

Loading cached processed dataset at /Users/herve/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb/cache-640662018a3bdca4.arrow
100%|██████████| 20/20 [00:06<00:00,  2.96ba/s]
Loading cached processed dataset at /Users/herve/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb/cache-8fc7790614434712.arrow


{'train': (160000, 4), 'validation': (20000, 4), 'test': (20000, 4)}

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("camembert-base", num_labels=2)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.weig

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [11]:
training_args = TrainingArguments("test_trainer")

In [12]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset, compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()